In [ ]:
## Libraries
import cv2
import os
import subprocess
import json
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import io
!pip install pymp4
from pymp4.parser import Box

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.7/717.7 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for construct: filename=construct-2.8.8-py2.py3-none-any.whl size=69821 sha256=b46043a31841e5a7b401395b271600f01ef0276a657c5c39164bb07e0fb74ce3
  Stored in directory: /root/.cache/pip/wheels/33/74/11/b423c36a4849492351928ee5720a641db40ff6ccca302144b1
Successfully built construct


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def parse_boxes(file):
    boxes = []
    while True:
        start_pos = file.tell()
        try:
            box = Box.parse_stream(file)
            try:
              box.data=0
            except:
              pass
            boxes.append(box)
            try:
              child_size_limit = start_pos + box.size
              box.children = parse_boxes(file, child_size_limit)
            except:
              pass
        except:
            break
    return boxes


# Knowledge-Based


In [ ]:
import string

In [ ]:
def check_key_exist(boxes, key):
  for box in boxes:
    if hasattr(box, key):
      return True
    if hasattr(box, 'children') and box.children:
      if check_key_exist(box.children, key):
        return True
    if hasattr(box, 'entries') and box.entries:
      if check_key_exist(box.entries, key):
        return True
    if hasattr(box, 'flags') and hasattr(box.flags, key):
        return True
    if hasattr(box, 'opcolor') and hasattr(box.opcolor, key):
        return True
    if hasattr(box, 'avc_data') and hasattr(box.avc_data, key):
        return True
    if hasattr(box, 'data_entries') and box.data_entries:
      if check_key_exist(box.data_entries, key):
        return True

  return False

In [ ]:
def find_value(boxes, key):
  for box in boxes:
    if hasattr(box, key):
      return getattr(box, key)
    if hasattr(box, 'children') and box.children:
      res = find_value(box.children, key)
      if res is not None:
        return res
    if hasattr(box, 'entries') and box.entries:
      res = find_value(box.entries, key)
      if res is not None:
        return res
    if hasattr(box, 'flags') and hasattr(box.flags, key):
        return getattr(box.flags, key)
    if hasattr(box, 'opcolor') and hasattr(box.opcolor, key):
        return getattr(box.opcolor, key)
    if hasattr(box, 'avc_data') and hasattr(box.avc_data, key):
        return getattr(box.avc_data, key)
    if hasattr(box, 'data_entries') and box.data_entries:
      res = find_value(box.data_entries, key)
      if res is not None:
        return res

  return None

In [ ]:
def find_box_has_key(boxes, key):
  for box in boxes:
    if hasattr(box, key):
      return box
    if hasattr(box, 'children') and box.children:
      res = find_box_has_key(box.children, key)
      if res is not None:
        return res
    if hasattr(box, 'entries') and box.entries:
      res = find_box_has_key(box.entries, key)
      if res is not None:
        return res
    if hasattr(box, 'flags') and hasattr(box.flags, key):
        return box.flags
    if hasattr(box, 'opcolor') and hasattr(box.opcolor, key):
        return box.opcolor
    if hasattr(box, 'avc_data') and hasattr(box.avc_data, key):
        return box.avc_data
    if hasattr(box, 'data_entries') and box.data_entries:
      res = find_box_has_key(box.data_entries, key)
      if res is not None:
        return res

  return None

In [ ]:
def find_box_size_has_name_alike(boxes, name):
  name = name.lower()
  for box in boxes:
    if hasattr(box, 'type'):
      if name in box.type.decode().lower():
        return box.end - box.offset
    if hasattr(box, 'children') and box.children:
      res = find_box_size_has_name_alike(box.children, name)
      if res is not None:
        return res

  return None

In [ ]:
def count_of_box(boxes, name):
  if type(name) != bytes:
    name = name.encode()
  count = 0
  for box in boxes:
    if hasattr(box, 'type'):
      if box.type == name:
        count += 1
    if hasattr(box, 'children') and box.children:
      count += count_of_box(box.children, name)
    if hasattr(box, 'entries') and box.entries:
      count += count_of_box(box.entries, name)
    if hasattr(box, 'flags') and hasattr(box.flags, 'type'):
      if box.flags.type == name:
        count += 1
    if hasattr(box, 'opcolor') and hasattr(box.opcolor, 'type'):
      if box.opcolor.type == name:
        count += 1
    if hasattr(box, 'avc_data') and hasattr(box.avc_data, 'type'):
      if box.avc_data.type == name:
        count += 1
    if hasattr(box, 'data_entries') and box.data_entries:
      count += count_of_box(box.data_entries, name)
  return count

In [ ]:
def find_vide_box(boxes):
  for box in boxes:
    if box.type == b'trak' and find_value([box], 'handler_type') == b'vide':
      return box
    if hasattr(box, 'children') and box.children:
      res = find_vide_box(box.children)
      if res is not None:
        return res
  return None

def find_soun_box(boxes):
  for box in boxes:
    if box.type == b'trak' and find_value([box], 'handler_type') == b'soun':
      return box
    if hasattr(box, 'children') and box.children:
      res = find_soun_box(box.children)
      if res is not None:
        return res
  return None

In [ ]:
def get_atom_names(boxes):
  atoms = []
  for box in boxes:
    if hasattr(box, 'type'):
      atoms.append(box.type.decode())
    if hasattr(box, 'children') and box.children:
      atoms += get_atom_names(box.children)

  return atoms

In [ ]:
def count_trak_type(boxes, trak_type):
  count = 0
  trak_type = trak_type.lower()
  for box in boxes:
    if box.type == b'trak' and trak_type in find_value([box], 'name').lower() :
      count+=1
    if hasattr(box, 'children') and box.children:
      count += count_trak_type(box.children, trak_type)
  return count

def count_unkown_trak_types(boxes, trak_types):
  count = 0
  trak_types = [i.lower() for i in trak_types]
  for box in boxes:
    if box.type == b'trak' and not any([(i in find_value([box], 'name').lower()) for i in trak_types]):
      count+=1
    if hasattr(box, 'children') and box.children:
      count += count_unkown_trak_types(box.children, trak_types)
  return count

In [ ]:
def count_handler_type(boxes, h_type):
  count = 0
  if type(h_type) != bytes:
    h_type = h_type.encode()
  for box in boxes:
    if box.type == b'hdlr' and h_type == box.handler_type :
      count+=1
    if hasattr(box, 'children') and box.children:
      count += count_handler_type(box.children, h_type)
  return count

def count_unkown_handler_types(boxes, h_types):
  count = 0
  if type(h_types[0]) != bytes:
    h_types = [i.encode() for i in h_types]
  for box in boxes:
    if box.type == b'hdlr' and not any([i == box.handler_type for i in h_types]):
      count+=1
    if hasattr(box, 'children') and box.children:
      count += count_unkown_handler_types(box.children, h_types)
  return count

In [ ]:
def knowledge_based_FE(file_path, label):
  with open(file_path, 'rb') as f:
    boxes = parse_boxes(f)

  video_box = [find_vide_box(boxes)]
  sound_box = [find_soun_box(boxes)]
  features = []
  try:   features.append(boxes[-1].end)#file size
  except:   features.append(None)

  features.append(int(check_key_exist(boxes, 'matrix')))#MP4 Transformation Mabix
  features.append(int(check_key_exist(boxes, 'rotation')))#MP4 Rotation
  features.append(int(check_key_exist(boxes, 'volume')))#MP4 Preferred Volume
  features.append(int(check_key_exist(boxes, 'rate')))#MP4 Preferred Rate
  features.append(int(check_key_exist(boxes, 'next_track_ID')))#MP4 Next Track ID
  features.append(int(check_key_exist(boxes, 'modification_time')))#MP4 Modification Time
  features.append(int(check_key_exist(boxes, 'minor_version')))#MP4 Minor Version
  features.append(int(check_key_exist(boxes, 'timescale')))#MP4 Media Time Scale
  features.append(int(check_key_exist(boxes, 'major_brand')))#MP4 Major Brand

  mb = bool(features[-1])
  if mb:
    major_brand = find_value(boxes, 'major_brand').decode().lower()
    features.append(int('3g2a' in major_brand))#MP4 Major Brand 3 g2.a
    features.append(int('3g2b' in major_brand))#MP4 Major Brand 3 g2.b
    features.append(int('3g2c' in major_brand))#MP4 Major Brand 3g2c
    features.append(int('3g06' in major_brand))#MP4 Major Brand 3 g06
    features.append(int('3p7' in major_brand))#MP4 Major Brand 3p7.
#5
    features.append(int('3gg6' in major_brand))#MP4 Major Brand 3 g.g6
    features.append(int('3gpl' in major_brand))#MP4 Major Brand 3gpl
    features.append(int('3gp2' in major_brand))#MP4 Major Brand 3gp2
    features.append(int('3gp3' in major_brand))#MP4 Major Brand 3gp3
    features.append(int('3gp4' in major_brand))#MP4 Major Brand 3gp4
#10
    features.append(int('3gp5' in major_brand))#MP4 Major Brand 3gp5
    features.append(int('3gp6' in major_brand))#MP4 Major Brand 3gp6
    features.append(int('3ss7' in major_brand))#MP4 Major Brand 3ss7
    features.append(int('avel' in major_brand))#MP4 Major Brand avel
    features.append(int('caep' in major_brand))#MP4 Major Brand CAEP
#15
    features.append(int('caqy' in major_brand))#MP4 Major Brand caqy
    features.append(int('cdes' in major_brand))#MP4 Major Brand CDes
    features.append(int('daoa' in major_brand))#MP4 Major Brand daOa
    features.append(int('daob' in major_brand))#MP4 Major Brand daOb
    features.append(int('dala' in major_brand))#MP4 Major Brand dala
#20
    features.append(int('dalb' in major_brand))#MP4 Major Brand dalb
    features.append(int('da2a' in major_brand))#MP4 Major Brand da2a
    features.append(int('da2b' in major_brand))#MP4 Major Brand_da2b
    features.append(int('da3a' in major_brand))#MP4 Major Brand da3a
    features.append(int('da3b' in major_brand))#MP4 Major Brand da3b
#25
    features.append(int('dmbl' in major_brand))#MP4 Major Brand dmbl
    features.append(int('dmpf' in major_brand))#MP4 Major Brand dmpf
    features.append(int('drel' in major_brand))#MP4 Major Brand drel
    features.append(int('dyla' in major_brand))#MP4 Major Brand dyla
    features.append(int('dvlb' in major_brand))#MP4 Major Brand dvlb
#30
    features.append(int('dv2a' in major_brand))#MP4 Major Brand dv2a
    features.append(int('dv2b' in major_brand))#MP4 Major Brand dv2b
    features.append(int('dv3a' in major_brand))#MP4 Major Brand dv3a
    features.append(int('dv3b' in major_brand))#MP4 Major Brand dv3b
    features.append(int('dvrl' in major_brand))#MP4 Major Brand dvrl
#35
    features.append(int('dvtl' in major_brand))#MP4 Major Brand dvtl
    features.append(int('f4v' in major_brand))#MP4 Major Brand F4V
    features.append(int('f4p' in major_brand))#MP4 Major Brand F4P
    features.append(int('f4a' in major_brand))#MP4 Major Brand F4A
    features.append(int('f48' in major_brand))#MP4 Major Brand F48
#40
    features.append(int('isc2' in major_brand))#MP4 Major Brand isc2.
    features.append(int('iso2' in major_brand))#MP4 Major Brand iso2
    features.append(int('isom' in major_brand))#MP4 Major Brand isom
    features.append(int('jp2' in major_brand))#MP4 Maior Brand JP2
    features.append(int('jp20' in major_brand))#MP4 Major Brand JP20
#45
    features.append(int('jpm' in major_brand))#MP4 Major Brand jpm
    features.append(int('ipx' in major_brand))#MP4 Major Brand ipx
    features.append(int('kddi' in major_brand))#MP4 Major Brand KDDI
    features.append(int('m4a' in major_brand))#MP4 Major Brand M4A
    features.append(int('m4b' in major_brand))#MP4 Major Brand M4B
#50
    features.append(int('m4p' in major_brand))#MP4 Major Brand M4P
    features.append(int('m4v' in major_brand))#MP4 Major Brand M4V
    features.append(int('m4vh' in major_brand))#MP4 Major Brand M4VH
    features.append(int('m4vp' in major_brand))#MP4 Major Brand M4VP
    features.append(int('mj2s' in major_brand))#MP4 Major Brand mj2s
#55
    features.append(int('mip2' in major_brand))#MP4 Major Brand mip2
    features.append(int('mmp4' in major_brand))#MP4 Major Brand mmp4
    features.append(int('mp21' in major_brand))#MP4 Major Brand mp21
    features.append(int('mp41' in major_brand))#MP4 Major Brand mp41
    features.append(int('mp42' in major_brand))#MP4 Major Brand mp42
#60
    features.append(int('mp71' in major_brand))#MP4 Major Brand mp71
    features.append(int('mppi' in major_brand))#MP4 Major Brand MPPI
    features.append(int('mat' in major_brand))#MP4 Major Brand mat
    features.append(int('msnv' in major_brand))#MP4 Maior Brand MSNV
    features.append(int('ndas' in major_brand))#MP4 Major Brand NDAS
#65
    features.append(int('ndsc' in major_brand))#MP4 Major Brand NDSC
    features.append(int('nosh' in major_brand))#MP4 Major Brand NOSH
    features.append(int('ndsm' in major_brand))#MP4 Major Brand NDSM
    features.append(int('ndsp' in major_brand))#MP4 Major Brand NDSP
    features.append(int('noss' in major_brand))#MP4 Major Brand NOSS
#70
    features.append(int('ndxc' in major_brand))#MP4 Major Brand NDXC
  else:
    for i in range(71): features.append(None)

  features.append(int(check_key_exist(sound_box, 'creation_time')))#MP4 Sound Creation Time
  features.append(int(check_key_exist(sound_box, 'balance')))#MP4 Sound Balance
  if mb:
    features.append(int('ndxh' in major_brand))#MP4 Major Brand NDXH
    features.append(int('ndxm' in major_brand))#MP4 Major Brand NDXM
    features.append(int('ndxp' in major_brand))#MP4 Major Brand NDXP
    features.append(int('ndxs' in major_brand))#MP4 Major Brand NDXS
    features.append(int('odd' in major_brand))#MP4 Major Brand odd
#5
    features.append(int('opf2' in major_brand))#MP4 Major Brand opf2
    features.append(int('opx2' in major_brand))#MP4 Major Brand opx2
    features.append(int('pana' in major_brand))#MP4 Major Brand pana
    features.append(int('qt' in major_brand))#MP4 Maior Brand qt
    features.append(int('ross' in major_brand))#MP4 Major Brand ROSS
#10
    features.append(int('sdv' in major_brand))#MP4 Major Brand sdv
    features.append(int('sscl' in major_brand))#MP4 Major Brand sscl
    features.append(int('ssc2' in major_brand))#MP4 Major Brand ssc2
    features.append(int((not any(bool(e) for e in features[-13:])) and
     (not any(bool(e) for e in features[-13-2-71:-13-2]))))#MP4 Major Brand Else
  else:
    for i in range(14): features.append(None)

  try:
    features.append(find_value(boxes,'duration')/1000)#MP4 Duration
  except:
    features.append(None)

  features.append(int(check_key_exist(boxes,'creation_time')))#MP4 Creation Time

  try:
    features.append(len(find_value(boxes,'compatible_brands'))*4)#MP4 Compatible Brands
  except:
    features.append(None)

  features.append(find_value(video_box, 'width'))#MP4 Video Width
  features.append(find_value(video_box,'vertical_resolution'))#MP4 Video Vertical Resolution

  try:
    features.append(int(list(find_value(video_box, 'opcolor')) == ['red', 'green', 'blue']))#MP4 Video Opcolor
  except:
    features.append(None)

  features.append(int(check_key_exist(video_box,'modification_time')))#MP4 Video Modification Time
  features.append(int(check_key_exist(video_box,'language')))#MP4 Video ISO 639-2 language Code

  lan = bool(features[-1])
  language = find_value(video_box,'language')
  if lan:
    features.append(int(language == 'eng'))#MP4 Video ISO 639 2 language Code Eng
    features.append(int(language == 'mis'))#MP4 VKleo ISO 639 2 language Code Mis
    features.append(int(language == 'und'))#MP4 Video ISO 639 2 language Code Und
    features.append(int(language == 'zxx'))#MP4 Video ISO 639 2 language Code Zxx
    features.append(int(language == 'qaa-qtz'))#MP4 Video ISO 639 2 language Code Qaa Qtz
#5
    features.append(int(all(c not in string.printable for c in language)))#MP4 Video ISO 639 2 language Code Gibberish
    features.append(int(not any(bool(e) for e in features[-6:])))#MP4 Video ISO 639 2 language Code Else
  else:
    for i in range(7): features.append(None)

  features.append(find_value(video_box,'horizontal_resolution'))#MP4 Video Horizontal Resolution
  features.append(find_value(video_box,'height'))#MP4 Video Height

  graph = check_key_exist(boxes, 'graphics_mode')
  features.append(int(graph))#MP4 Video GraphK: s Mode
  if graph:
    graphics_mode = find_value(boxes, 'graphics_mode')
    features.append(int(graphics_mode == 'copy'))#MP4 Video GraphK: s Mode Copy
    features.append(int(graphics_mode == 'ditherCopy'))#MP4 Video GraphK: s Mode ditherCopy
    features.append(int(graphics_mode == 'Wend'))#MP4 Video GraphK: s Mode Wend
    features.append(int(graphics_mode == 'Transparent'))#MP4 Video GraphK: s Mode Transparent
    features.append(int(graphics_mode == 'SlraightAlpha'))#MP4 Video GraphK: s Mode SlraightAlpha
#5
    features.append(int(graphics_mode == 'sreCopy'))#MP4 Video GraphK: s Mode sreCopy
    features.append(int(graphics_mode == 'PremulWhiteAlpha'))#MP4 Video GraphK: s Mode PremulWhiteAlpha
    features.append(int(graphics_mode == 'Premulblackalpha'))#MP4 Video GraphK: s Mode Premulblackalpha
    features.append(int(graphics_mode == 'Straightalphablend'))#MP4 Video GraphK: s Mode Straightalphablend
    features.append(int(graphics_mode == 'Composition'))#MP4 Video Graphics Mode Composition
#10
    features.append(int(not any(bool(e) for e in features[-10:])))#MP4 Video Graphics Mode Else
  else:
    for i in range(11): features.append(None)

  features.append(find_value(video_box,'frame_count'))#MP4 Video Frame Rate
  features.append(find_value(video_box,'depth'))#MP4 Video Depth
  features.append(int(check_key_exist(video_box,'creation_time')))#MP4 Video Creation Time

  compressor_box = [find_box_has_key(video_box,'compressor_name')]
  features.append(int(compressor_box[0] != None))#MP4 Video Compressor Name
  if bool(features[-1]):
    compressor_name = find_value(compressor_box,'compressor_name').decode().lower()
    features.append(int('avc' in compressor_name))#MP4 Video Compressor Name AVC
    features.append(int('hevc' in compressor_name))#MP4 Video Compressor Name HEVC
    features.append(int('h 264' in compressor_name))#MP4 Video Compressor Name H 264
    features.append(int(not any(bool(e) for e in features[-3:])))#MP4 Video Compressor Name Else
  else:
    for i in range(4): features.append(None)

  features.append(int(check_key_exist(compressor_box,'type')))#MP4 Video Compression Type

  ct = features[-1]
  if ct:
    compression_type = find_value(compressor_box,'type').decode().upper()
    features.append(int('H261' in compression_type))#MP4 Video Compression Type H 261
    features.append(int('H263'in compression_type))#MP4 Video Compression Type H 263
    features.append(int('H264' in compression_type))#MP4 Video Compression Type H 264
    features.append(int('H265' in compression_type))#MP4 Video Compression Type H 265
    features.append(int('HEVC' in compression_type))#MP4 Video Compression Type HEVC
#5
    features.append(int('MPEG' in compression_type))#MP4 Video Compression Type MPEG
    features.append(int('WMV' in compression_type))#MP4 Video Compression Type WMV
    features.append(int('OIVX' in compression_type))#MP4 Video Compression Type OivX
    features.append(int('AVI' in compression_type))#MP4 Video Compression Type AVI
    features.append(int('AVC' in compression_type))#MP4 Video Compression Type AVC
#10
    features.append(int(not any(bool(e) for e in features[-10:])))#MP4 Video Compression Type Else
  else:
    for i in range(11): features.append(None)

  features.append(find_value(sound_box, 'bits_per_sample'))#MP4 Sound Sample Size
  features.append(find_value(sound_box,'sample_count'))#MP4 Sound Sample Rate
  features.append(find_value(sound_box,'channels'))#MP4 Sound Number of Channels
  features.append(int(check_key_exist(sound_box,'modification_time')))#MP4 Sound Modification Time
  features.append(int(check_key_exist(sound_box,'language')))#MP4 Sound ISO 639 2 languap Code

  lan = bool(features[-1])
  language_ = find_value(sound_box,'language')
  if lan:
    features.append(int(language_ == 'eng'))#MP4 Sound ISO 639 2 languap Code Eng
    features.append(int(language_ == 'mis'))#MP4 Sound ISO 639 2 language Code Mis
    features.append(int(language_ == 'und'))#MP4 Sound ISO 639 2 language Code Und
    features.append(int(language_ == 'zxx'))#MP4 Sound ISO 639 2 languap Code Zxx
    features.append(int(language_ == 'qaa-qtz'))#MP4 Sound ISO 639 2 languap Code Qaa Qtz
#5
    features.append(int(all(c not in string.printable for c in language_)))#MP4 Sound ISO 639 2 language Code Gibberish
    features.append(int(not any(bool(e) for e in features[-6:])))#MP4 Sound ISO 639 2 language Code Else
  else:
    for i in range(7): features.append(None)

  features.append(int(check_key_exist(sound_box,'format')))#MP4 Sound Format

  f = bool(features[-1])
  if f:
    format = find_value(sound_box,'format').decode()
    features.append(int(format == 'aa:.'))#MP4 Sound Format Aa:.
    features.append(int(not features[-1]))#MP4 Sound Format Else
  else:
    for i in range(2): features.append(None)

  features.append(int(language_==language))#MP4 Sound Video language Code IsEqual

  try:
    features.append(find_value(video_box,'height')*find_value(video_box,'width'))#MP4 Video ImageSize
  except:
    features.append(None)

  features.append(count_of_box(boxes, 'url '))#Number of Uris
  features.append(count_of_box(boxes, 'ip  '))#IP Count
  features.append(count_of_box(boxes, 'emai'))#EmailAddress Count

  with open(file_path, 'rb') as f:
    full = f.read()
    qlen = len(full)//4
    q1 = full[:qlen]
    q2 = full[qlen:2*qlen]
    q3 = full[2*qlen:3*qlen]
    q4 = full[3*qlen:]
    with open('q1.txt', 'wb') as f:
        f.write(q1)
    with open('q2.txt', 'wb') as f:
        f.write(q2)
    with open('q3.txt', 'wb') as f:
        f.write(q3)
    with open('q4.txt', 'wb') as f:
        f.write(q4)

  with open('q1.txt', 'rb') as f:
    lines = f.readlines()
    lines = [len(line) for line in lines]
    features.append(np.mean(lines))#FirstQ.uerterAsTextMean
    features.append(np.std(lines))#FirstQ, uerterAsText: Sd

  with open('q2.txt', 'rb') as f:
    lines = f.readlines()
    lines = [len(line) for line in lines]
    features.append(np.mean(lines))#SemndQuerterAsTextMean
    features.append(np.std(lines))#SemndQuerterAsTextSd

  with open('q3.txt', 'rb') as f:
    lines = f.readlines()
    lines = [len(line) for line in lines]
    features.append(np.mean(lines))#ThirdQuerterAsTextMean
    features.append(np.std(lines))#ThirdQuerterAsTextSd

  with open('q4.txt', 'rb') as f:
    lines = f.readlines()
    lines = [len(line) for line in lines]
    features.append(np.mean(lines))#ForthQuerterAsTextMean
    features.append(np.std(lines))#ForthQuerterAsTextSd ~ Forth Quarter As Text Sd

  features.append(int(subprocess.run(["ffmpeg", "-i", file_path, '-f', 'null','-'], stderr=subprocess.PIPE).returncode != 0))#ExceptionOuring Runtime ~ Exception During Runtime
  features.append(find_box_size_has_name_alike(boxes,'alb'))#alb size
  features.append(find_box_size_has_name_alike(boxes,'ART'))#ART size
  features.append(find_box_size_has_name_alike(boxes,'cmt'))#cmt size
  features.append(find_box_size_has_name_alike(boxes,'day'))#day size
  features.append(find_box_size_has_name_alike(boxes,'nam'))#nam size
  features.append(find_box_size_has_name_alike(boxes,'too'))#too size
  features.append(find_box_size_has_name_alike(boxes,'wrt'))#wrt_size
  features.append(find_box_size_has_name_alike(boxes,'covr'))#covr size
  features.append(find_box_size_has_name_alike(boxes,'etts'))#etts size
  features.append(find_box_size_has_name_alike(boxes,'data'))#data size
  features.append(find_box_size_has_name_alike(boxes,'desc'))#desc size
  features.append(find_box_size_has_name_alike(boxes,'dinf'))#dinf size
  features.append(find_box_size_has_name_alike(boxes,'dref'))#dref size
  features.append(find_box_size_has_name_alike(boxes,'edts'))#edts size
  features.append(find_box_size_has_name_alike(boxes,'elst'))#elst size
  features.append(find_box_size_has_name_alike(boxes,'free'))#free size
  features.append(find_box_size_has_name_alike(boxes,'ftyp'))#ftyp size
  features.append(find_box_size_has_name_alike(boxes,'hdlr'))#hdlr size
  features.append(find_box_size_has_name_alike(boxes,'ilst'))#ilst size
  features.append(find_box_size_has_name_alike(boxes,'iods'))#iods size
  features.append(find_box_size_has_name_alike(boxes,'mdat'))#mdat size
  features.append(find_box_size_has_name_alike(boxes,'mdhd'))#mdhd size
  features.append(find_box_size_has_name_alike(boxes,'mdia'))#mdia size
  features.append(find_box_size_has_name_alike(boxes,'meta'))#meta size
  features.append(find_box_size_has_name_alike(boxes,'minf'))#minf size
  features.append(find_box_size_has_name_alike(boxes,'moov'))#moov size
  features.append(find_box_size_has_name_alike(boxes,'mvhd'))#mvhd size
  features.append(find_box_size_has_name_alike(boxes,'nled'))#nled size
  features.append(find_box_size_has_name_alike(boxes,'nmhd'))#nmhd size
  features.append(find_box_size_has_name_alike(boxes,'sbep'))#sbep size
  features.append(find_box_size_has_name_alike(boxes,'sdtp'))#sdtp size
  features.append(find_box_size_has_name_alike(boxes,'sgpd'))#sgpd size
  features.append(find_box_size_has_name_alike(boxes,'smhd'))#smhd size
  features.append(find_box_size_has_name_alike(boxes,'size'))#size
  features.append(find_box_size_has_name_alike(boxes,'stco'))#stco size
  features.append(find_box_size_has_name_alike(boxes,'stse'))#stse size
  features.append(find_box_size_has_name_alike(boxes,'stsd'))#stsd size
  features.append(find_box_size_has_name_alike(boxes,'stss'))#stss size
  features.append(find_box_size_has_name_alike(boxes,'stsz'))#stsz size
  features.append(find_box_size_has_name_alike(boxes,'stts'))#stts size
  features.append(find_box_size_has_name_alike(boxes,'tkhd'))#tkhd size
  features.append(find_box_size_has_name_alike(boxes,'trak'))#trak size
  features.append(find_box_size_has_name_alike(boxes,'udta'))#udta size
  features.append(find_box_size_has_name_alike(boxes,'vmhd'))#vmhd size
  features.append(find_box_size_has_name_alike(boxes,'uuid'))#uuid size

  atoms = get_atom_names(boxes)
  u_count=0
  for a in atoms:
    if a[0] not in ['alb','ART','cmt','day','nam','too','wrt','covr','etts','data','desc',
                    'dinf','dref','edts','elst','free','ftyp','hdlr','ilst','iods','mdat',
                    'mdhd','mdia','meta','minf','moov','mvhd','nled','nmhd','sbep','sdtp',
                    'sgpd','smhd','size','stco','stse','stsd','stss','stsz','stts','tkhd',
                    'trak','udta','vmhd','uuid']:
      u_count+=1
  features.append(u_count)#nottnAtomUst count

  uuid_count=0
  for n in atoms:
    if n=='uuid':
      uuid_count+=1
  features.append(uuid_count)#uuid count

  data_count=0
  for n in atoms:
    if n=='data':
      data_count+=1
  features.append(data_count)#data count

  features.append(count_unkown_trak_types(boxes,['video','sound','hint','system']))#Unknown_track_count
  features.append(count_trak_type(boxes,'sound'))#Audio track count
  features.append(count_trak_type(boxes,'video'))#Video track count
  features.append(count_trak_type(boxes,'hint'))#Hint_track_count
  features.append(count_trak_type(boxes,'system'))#Systems track count
  features.append(count_unkown_handler_types(boxes, ['soun','vide','odsm','sdsm','m7sm',
                                                     'ocsm','ipsm','mism','hint','crsm']))#Unknown Handler count
  features.append(count_handler_type(boxes,'soun'))#soun Handler count
  features.append(count_handler_type(boxes,'vide'))#vide Handler count
  features.append(count_handler_type(boxes,'odsm'))#odsm Handler count
  features.append(count_handler_type(boxes,'sdsm'))#sdsm Handler count
  features.append(count_handler_type(boxes,'m7sm'))#m7sm Handler count
  features.append(count_handler_type(boxes,'ocsm'))#ocsm Handler count
  features.append(count_handler_type(boxes,'ipsm'))#ipsm Handler count
  features.append(count_handler_type(boxes,'mism'))#mism Handler count
  features.append(count_handler_type(boxes,'hint'))#hint Handler count
  features.append(count_handler_type(boxes,'crsm'))#crsm Handler count

  with open(file_path, 'rb') as f:
    str_data = str(f.read(), 'latin1')
  features.append(str_data.count('bin/sh'))#binSh count
  features.append(str_data.count('system'))#system count
  features.append(str_data.count('wix'))#wix count

  other = ['dir','cd','mkdir','rmdir','del','copy','move','systeminfo','tasklist','taskkill',
           'ipconfig','ping','tracert','netstat','nslookup','chkdsk','diskpart','format','echo',
           'cls','type','find','fc','help','.bat','powershell','python','.exe','java','.js','start']
  count = 0
  for o in other:
    count += str_data.count(o)
  features.append(count)#other count

  features.append(label)
  return features

In [ ]:
features = ['file size', 'MP4 Transformation Mabix', 'MP4 Rotation', 'MP4 Preferred Volume',
            'MP4 Preferred Rate', 'MP4 Next Track ID', 'MP4 Modification Time', 'MP4 Minor Version',
            'MP4 Media Time Scale', 'MP4 Major Brand', 'MP4 Major Brand 3 g2.a', 'MP4 Major Brand 3 g2.b',
            'MP4 Major Brand 3g2c', 'MP4 Major Brand 3 g06', 'MP4 Major Brand 3p7.', 'MP4 Major Brand 3 g.g6',
            'MP4 Major Brand 3gpl', 'MP4 Major Brand 3gp2', 'MP4 Major Brand 3gp3', 'MP4 Major Brand 3gp4',
            'MP4 Major Brand 3gp5', 'MP4 Major Brand 3gp6', 'MP4 Major Brand 3ss7', 'MP4 Major Brand avel',
            'MP4 Major Brand CAEP', 'MP4 Major Brand caqy', 'MP4 Major Brand CDes', 'MP4 Major Brand daOa',
            'MP4 Major Brand daOb', 'MP4 Major Brand dala', 'MP4 Major Brand dalb', 'MP4 Major Brand da2a',
            'MP4 Major Brand_da2b', 'MP4 Major Brand da3a', 'MP4 Major Brand da3b', 'MP4 Major Brand dmbl',
            'MP4 Major Brand dmpf', 'MP4 Major Brand drel', 'MP4 Major Brand dyla', 'MP4 Major Brand dvlb',
            'MP4 Major Brand dv2a', 'MP4 Major Brand dv2b', 'MP4 Major Brand dv3a', 'MP4 Major Brand dv3b',
            'MP4 Major Brand dvrl', 'MP4 Major Brand dvtl', 'MP4 Major Brand F4V', 'MP4 Major Brand F4P',
            'MP4 Major Brand F4A', 'MP4 Major Brand F48', 'MP4 Major Brand isc2.', 'MP4 Major Brand iso2',
            'MP4 Major Brand isom', 'MP4 Maior Brand JP2', 'MP4 Major Brand JP20', 'MP4 Major Brand jpm',
            'MP4 Major Brand ipx', 'MP4 Major Brand KDDI', 'MP4 Major Brand M4A', 'MP4 Major Brand M4B',
            'MP4 Major Brand M4P', 'MP4 Major Brand M4V', 'MP4 Major Brand M4VH', 'MP4 Major Brand M4VP',
            'MP4 Major Brand mj2s', 'MP4 Major Brand mip2', 'MP4 Major Brand mmp4', 'MP4 Major Brand mp21',
            'MP4 Major Brand mp41', 'MP4 Major Brand mp42', 'MP4 Major Brand mp71', 'MP4 Major Brand MPPI',
            'MP4 Major Brand mat', 'MP4 Maior Brand MSNV', 'MP4 Major Brand NDAS', 'MP4 Major Brand NDSC',
            'MP4 Major Brand NOSH', 'MP4 Major Brand NDSM', 'MP4 Major Brand NDSP', 'MP4 Major Brand NOSS',
            'MP4 Major Brand NDXC', 'MP4 Sound Creation Time', 'MP4 Sound Balance', 'MP4 Major Brand NDXH',
            'MP4 Major Brand NDXM', 'MP4 Major Brand NDXP', 'MP4 Major Brand NDXS', 'MP4 Major Brand odd',
            'MP4 Major Brand_opf2', 'MP4 Major Brand opx2', 'MP4 Major Brand pana', 'MP4 Maior Brand qt',
            'MP4 Major Brand ROSS', 'MP4 Major Brand sdv', 'MP4 Major Brand sscl', 'MP4 Major Brand ssc2',
            'MP4 Major Brand Else', 'MP4 Duration', 'MP4 Creation Time', 'MP4 Compatible Brands', 'MP4 Video Width',
            'MP4 Video Vertical Resolution', 'MP4 Video Opcolor', 'MP4 Video Modification Time', 'MP4 Video ISO 639 2 language Code',
            'MP4 Video ISO 639 2 language Code Eng', 'MP4 VKleo ISO 639 2 language Code Mis', 'MP4 Video ISO 639_2_language Code Und',
            'MP4 Video ISO 639 2 language Code Zxx', 'MP4 Video ISO 639 2 language Code Qaa Otz', 'MP4 Video ISO 639 2 language Code Gibberish',
            'MP4 Video ISO 639 2 language Code Else', 'MP4 Video Horizontal Resolution', 'MP4 Video Height',
            'MP4 Video GraphK: s Mode', 'MP4 Video GraphK: s Mode Copy', 'MP4 Video GraphK: s Mode ditherCopy', 'MP4 Video GraphK: s Mode Wend',
            'MP4 Video GraphK: s Mode Transparent', 'MP4 Video GraphK: s Mode SlraightAlpha', 'MP4 Video GraphK: s Mode sreCopy',
            'MP4 Video GraphK: s Mode PremulWhiteAlpha', 'MP4 Video GraphK: s Mode Premulblackalpha', 'MP4 Video GraphK: s Mode Straightalphablend',
            'MP4 Video Graphics Mode Composition', 'MP4 Video Graphics Mode Else', 'MP4 Video Frame Rate',
            'MP4 Video Depth', 'MP4 Video Creation Time', 'MP4 Video Compressor Name', 'MP4 Video Compressor Name AVC',
            'MP4 Video Compressor Name HEVC', 'MP4 Video Compressor Name H 264', 'MP4 Video Compressor Name Else',
            'MP4 Video Compression Type', 'MP4 Video Compression Type H 261', 'MP4 Video Compression Type H 263', 'MP4 Video Compression Type H 264',
            'MP4 Video Compression Type H 265', 'MP4 Video Compression Type HEVC', 'MP4 Video Compression Type_MPEG',
            'MP4 Video Compression Type WMV', 'MP4 Video Compression Type OivX', 'MP4 Video Compression Type AVI',
            'MP4 Video Compression Type AVC', 'MP4 Video Compression Type Else', 'MP4 Sound Sample Size',
            'MP4 Sound Sample Rate', 'MP4 Sound Number of Channels', 'MP4 Sound Modification Time', 'MP4 Sound ISO 639 2 languap Code',
            'MP4 Sound ISO 639 2 languap Code Eng', 'MP4 Sound ISO 639 2 language Code Mis', 'MP4 Sound ISO 639 2 language Code Und',
            'MP4 Sound ISO 639 2 languap Code Zxx', 'MP4 Sound ISO 639 2 languap Code Qaa Qtz', 'MP4 Sound ISO 639 2 language Code Gibberish',
            'MP4 Sound ISO 639 2 language Code Else', 'MP4 Sound Format', 'MP4 Sound Format Aa:.', 'MP4 Sound Format Else',
            'MP4 Sound Video language Code IsEqual', 'MP4 Video ImageSize', 'Number of Uris', 'IP Count',
            'EmailAddress Count', 'FirstQ.uerterAsTextMean', 'FirstQ, uerterAsText: Sd', 'SemndQuerterAsTextMean',
            'SemndQuerterAsTextSd', 'ThirdQuerterAsTextMean', 'ThirdQuerterAsTextSd', 'ForthQuerterAsTextMean',
            'ForthQuerterAsTextSd', 'ExceptionOuring Runtime', 'alb size', 'ART size', 'cmt size',
            'day size', 'nam size', 'too size', 'wrt_size', 'covr size', 'etts size', 'data size',
            'desc size', 'dinf size', 'dref size', 'edts size', 'elst size', 'free size', 'ftyp size',
            'hdlr size', 'ilst size', 'iods size', 'mdat size', 'mdhd size', 'mdia size ', 'meta size',
            'minf size', 'moov size', 'mvhd size', 'nled size', 'nmhd size', 'sbep size', 'sdtp size',
            'sgpd size', 'smhd size', 'size', 'stco size', 'stse_size', 'stsd size', 'stss_size',
            'stsz size', 'stts size', 'tkhd size', 'trak size', 'udta size', 'vmhd size', 'uuid size',
            'nottnAtomUst count', 'uuid count', 'data count', 'Unknown_track_count', 'Audio track count',
            'Video track count', 'Hint_track_count', 'Systems track count', 'Unknown Handler count', 'soun Handler count',
            'vide Handler count', 'odsm Handler count', 'sdsm Handler count', 'm7sm Handler count', 'ocsm Handler count',
            'ipsm Handler count', 'mism Handler count', 'hint Handler count', 'crsm Handler count', 'binSh count',
            'system count', 'wix count', 'other count']

# Create dataset from features set
dataset_kb = pd.DataFrame(columns=[s for s in features] + ['IsMal'])   # IsMal is label 1 is malware, 0 is not malware

# Directory containing MP4 files
ok_directory = '/content/drive/MyDrive/nt230_collection/ok'
mal_directory = '/content/drive/MyDrive/nt230_collection/mal'

# Update dataset
for filename in os.listdir(mal_directory):
    if filename.endswith('.mp4'):
        file_path = os.path.join(mal_directory, filename)
        dataset_kb.loc[len(dataset_kb)] = knowledge_based_FE(file_path, 1)
for filename in os.listdir(ok_directory):
    if filename.endswith('.mp4'):
        file_path = os.path.join(ok_directory, filename)
        dataset_kb.loc[len(dataset_kb)] = knowledge_based_FE(file_path, 0)

In [ ]:
dataset_kb.to_csv('dataset_1.csv', index=False)

In [ ]:
os.listdir(mal_directory) + os.listdir(ok_directory)

['mal.mp4',
 'cve-2015-1538-1.mp4',
 'hevc-crash-poc.mp4',
 'videopoc.mp4',
 'ok.mp4',
 'ok1.mp4',
 'ok2.mp4',
 'ok3.mp4']

In [ ]:
print(dataset_kb.shape)
dataset_kb

(8, 244)


,file size,MP4 Transformation Mabix,MP4 Rotation,MP4 Preferred Volume,MP4 Preferred Rate,MP4 Next Track ID,MP4 Modification Time,MP4 Minor Version,MP4 Media Time Scale,MP4 Major Brand,MP4 Major Brand 3 g2.a,MP4 Major Brand 3 g2.b,MP4 Major Brand 3g2c,MP4 Major Brand 3 g06,MP4 Major Brand 3p7.,MP4 Major Brand 3 g.g6,MP4 Major Brand 3gpl,MP4 Major Brand 3gp2,MP4 Major Brand 3gp3,MP4 Major Brand 3gp4,MP4 Major Brand 3gp5,MP4 Major Brand 3gp6,MP4 Major Brand 3ss7,MP4 Major Brand avel,MP4 Major Brand CAEP,MP4 Major Brand caqy,MP4 Major Brand CDes,MP4 Major Brand daOa,MP4 Major Brand daOb,MP4 Major Brand dala,MP4 Major Brand dalb,MP4 Major Brand da2a,MP4 Major Brand_da2b,MP4 Major Brand da3a,MP4 Major Brand da3b,MP4 Major Brand dmbl,MP4 Major Brand dmpf,MP4 Major Brand drel,MP4 Major Brand dyla,MP4 Major Brand dvlb,MP4 Major Brand dv2a,MP4 Major Brand dv2b,MP4 Major Brand dv3a,MP4 Major Brand dv3b,MP4 Major Brand dvrl,MP4 Major Brand dvtl,MP4 Major Brand F4V,MP4 Major Brand F4P,MP4 Major Brand F4A,MP4 Major Brand F48,MP4 Major Brand isc2.,MP4 Major Brand iso2,MP4 Major Brand isom,MP4 Maior Brand JP2,MP4 Major Brand JP20,MP4 Major Brand jpm,MP4 Major Brand ipx,MP4 Major Brand KDDI,MP4 Major Brand M4A,MP4 Major Brand M4B,MP4 Major Brand M4P,MP4 Major Brand M4V,MP4 Major Brand M4VH,MP4 Major Brand M4VP,MP4 Major Brand mj2s,MP4 Major Brand mip2,MP4 Major Brand mmp4,MP4 Major Brand mp21,MP4 Major Brand mp41,MP4 Major Brand mp42,MP4 Major Brand mp71,MP4 Major Brand MPPI,MP4 Major Brand mat,MP4 Maior Brand MSNV,MP4 Major Brand NDAS,MP4 Major Brand NDSC,MP4 Major Brand NOSH,MP4 Major Brand NDSM,MP4 Major Brand NDSP,MP4 Major Brand NOSS,MP4 Major Brand NDXC,MP4 Sound Creation Time,MP4 Sound Balance,MP4 Major Brand NDXH,MP4 Major Brand NDXM,MP4 Major Brand NDXP,MP4 Major Brand NDXS,MP4 Major Brand odd,MP4 Major Brand_opf2,MP4 Major Brand opx2,MP4 Major Brand pana,MP4 Maior Brand qt,MP4 Major Brand ROSS,MP4 Major Brand sdv,MP4 Major Brand sscl,MP4 Major Brand ssc2,MP4 Major Brand Else,MP4 Duration,MP4 Creation Time,MP4 Compatible Brands,MP4 Video Width,MP4 Video Vertical Resolution,MP4 Video Opcolor,MP4 Video Modification Time,MP4 Video ISO 639 2 language Code,MP4 Video ISO 639 2 language Code Eng,MP4 VKleo ISO 639 2 language Code Mis,MP4 Video ISO 639_2_language Code Und,MP4 Video ISO 639 2 language Code Zxx,MP4 Video ISO 639 2 language Code Qaa Otz,MP4 Video ISO 639 2 language Code Gibberish,MP4 Video ISO 639 2 language Code Else,MP4 Video Horizontal Resolution,MP4 Video Height,MP4 Video GraphK: s Mode,MP4 Video GraphK: s Mode Copy,MP4 Video GraphK: s Mode ditherCopy,MP4 Video GraphK: s Mode Wend,MP4 Video GraphK: s Mode Transparent,MP4 Video GraphK: s Mode SlraightAlpha,MP4 Video GraphK: s Mode sreCopy,MP4 Video GraphK: s Mode PremulWhiteAlpha,MP4 Video GraphK: s Mode Premulblackalpha,MP4 Video GraphK: s Mode Straightalphablend,MP4 Video Graphics Mode Composition,MP4 Video Graphics Mode Else,MP4 Video Frame Rate,MP4 Video Depth,MP4 Video Creation Time,MP4 Video Compressor Name,MP4 Video Compressor Name AVC,MP4 Video Compressor Name HEVC,MP4 Video Compressor Name H 264,MP4 Video Compressor Name Else,MP4 Video Compression Type,MP4 Video Compression Type H 261,MP4 Video Compression Type H 263,MP4 Video Compression Type H 264,MP4 Video Compression Type H 265,MP4 Video Compression Type HEVC,MP4 Video Compression Type_MPEG,MP4 Video Compression Type WMV,MP4 Video Compression Type OivX,MP4 Video Compression Type AVI,MP4 Video Compression Type AVC,MP4 Video Compression Type Else,MP4 Sound Sample Size,MP4 Sound Sample Rate,MP4 Sound Number of Channels,MP4 Sound Modification Time,MP4 Sound ISO 639 2 languap Code,MP4 Sound ISO 639 2 languap Code Eng,MP4 Sound ISO 639 2 language Code Mis,MP4 Sound ISO 639 2 language Code Und,MP4 Sound ISO 639 2 languap Code Zxx,MP4 Sound ISO 639 2 languap Code Qaa Qtz,MP4 Sound ISO 639 2 language Code Gibberish,MP4 Sound ISO 639 2 language Code Else,MP4 Sound Format,MP4 Sound Format Aa:.,MP4 Sound Format Else,MP4 Sound

# Atom structural


In [ ]:
def update_atom_structs(atom_structs, boxes, parent='', verbose=False):
    for box in boxes:
        path = parent + f"/{box.type.decode()}"
        if(verbose): print(path)
        if(path not in atom_structs):
            atom_structs.append(path)
        if hasattr(box, 'children') and box.children:
            atom_structs = update_atom_structs(atom_structs, box.children, path, verbose=verbose)
    return atom_structs

def get_atom_structs(boxes, parent='', verbose=False, atom_structs=None):
    if atom_structs is None:
        atom_structs = []
    for box in boxes:
        path = parent + f"/{box.type.decode()}"
        if(verbose): print(path)
        atom_structs.append(path)
        if hasattr(box, 'children') and box.children:
            atom_structs = get_atom_structs(atom_structs = atom_structs, boxes = box.children, parent = path, verbose=verbose)
    return atom_structs

In [ ]:
def atom_structs_FE(file_name, IsMal, all_features, representation='count'):
  with open(file_name, 'rb') as f:
    boxes = parse_boxes(f)
  features = get_atom_structs(boxes)  #Extract feature from file
  if representation == 'count':
    return [features.count(feature) for feature in all_features] + [IsMal]
  if representation == 'bin':
    return [int(feature in features) for feature in all_features] + [IsMal]
  return None

In [ ]:
## Create features set for dataset
atom_structs = []

# Directory containing MP4 files
ok_directory = '/content/drive/MyDrive/nt230_collection/ok'
mal_directory = '/content/drive/MyDrive/nt230_collection/mal'

for filename in os.listdir(ok_directory):
    if filename.endswith('.mp4'):
        file_path = os.path.join(ok_directory, filename)
        # Updating the atom struct features
        with open(file_path, 'rb') as f:
            boxes = parse_boxes(f)
            atom_structs = update_atom_structs(atom_structs,boxes)
        print(f"Update atom structs features set by {file_path}")
for filename in os.listdir(mal_directory):
    if filename.endswith('.mp4'):
        file_path = os.path.join(mal_directory, filename)
        with open(file_path, 'rb') as f:
            boxes = parse_boxes(f)
            atom_structs = update_atom_structs(atom_structs,boxes)
        print(f"Update atom structs features set by {file_path}")

# Create dataset from features set
dataset_as = pd.DataFrame(columns=atom_structs + ['IsMal'])   # IsMal is label 1 is malware, 0 is not malware

# Update dataset
for filename in os.listdir(mal_directory):
    if filename.endswith('.mp4'):
        file_path = os.path.join(mal_directory, filename)
        dataset_as.loc[len(dataset_as)] = atom_structs_FE(file_path, 1, atom_structs)
for filename in os.listdir(ok_directory):
    if filename.endswith('.mp4'):
        file_path = os.path.join(ok_directory, filename)
        dataset_as.loc[len(dataset_as)] = atom_structs_FE(file_path, 0, atom_structs)

# Create dataset from features set
dataset_as_bin = pd.DataFrame(columns=atom_structs + ['IsMal'])   # IsMal is label 1 is malware, 0 is not malware

# Update dataset
for filename in os.listdir(mal_directory):
    if filename.endswith('.mp4'):
        file_path = os.path.join(mal_directory, filename)
        dataset_as_bin.loc[len(dataset_as_bin)] = atom_structs_FE(file_path, 1, atom_structs, representation='bin')
for filename in os.listdir(ok_directory):
    if filename.endswith('.mp4'):
        file_path = os.path.join(ok_directory, filename)
        dataset_as_bin.loc[len(dataset_as_bin)] = atom_structs_FE(file_path, 0, atom_structs, representation='bin')

Update atom structs features set by /content/drive/MyDrive/nt230_collection/ok/ok.mp4
Update atom structs features set by /content/drive/MyDrive/nt230_collection/ok/ok1.mp4
Update atom structs features set by /content/drive/MyDrive/nt230_collection/ok/ok2.mp4
Update atom structs features set by /content/drive/MyDrive/nt230_collection/ok/ok3.mp4
Update atom structs features set by /content/drive/MyDrive/nt230_collection/mal/mal.mp4
Update atom structs features set by /content/drive/MyDrive/nt230_collection/mal/cve-2015-1538-1.mp4
Update atom structs features set by /content/drive/MyDrive/nt230_collection/mal/hevc-crash-poc.mp4
Update atom structs features set by /content/drive/MyDrive/nt230_collection/mal/videopoc.mp4


In [ ]:
dataset_as.to_csv('dataset_2.csv', index=False)
dataset_as_bin.to_csv('dataset_3.csv', index=False)

In [ ]:
print(dataset_as.shape)
dataset_as

(8, 39)


,/ftyp,/uuid,/mdat,/moov,/moov/mvhd,/moov/trak,/moov/trak/tkhd,/moov/trak/mdia,/moov/trak/mdia/mdhd,/moov/trak/mdia/hdlr,/moov/trak/mdia/minf,/moov/trak/mdia/minf/vmhd,/moov/trak/mdia/minf/dinf,/moov/trak/mdia/minf/dinf/dref,/moov/trak/mdia/minf/stbl,/moov/trak/mdia/minf/stbl/stsd,/moov/trak/mdia/minf/stbl/stts,/moov/trak/mdia/minf/stbl/stsc,/moov/trak/mdia/minf/stbl/stsz,/moov/trak/mdia/minf/stbl/stco,/moov/trak/mdia/minf/stbl/stss,/moov/trak/mdia/minf/smhd,/moov/trak/mdia/minf/stbl/esds,/moov/trak/mdia/minf/stbl/ctts,/moov/udta,/free,/moov/trak/edts,/moov/trak/mdia/minf/stbl/sgpd,/moov/trak/mdia/minf/stbl/sbgp,/smss,/stsd,/stts,/stss,/ctts,/stsc,/stsz,/cprt,/,IsMal
0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1
1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1,1,1,1,1,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1,0,1,1,1,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,1,1,1,0,1,1,2,1,1,0,0,0,0,0,0,0,0,0,0
6,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
7,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
dataset_as_bin

,/ftyp,/uuid,/mdat,/moov,/moov/mvhd,/moov/trak,/moov/trak/tkhd,/moov/trak/mdia,/moov/trak/mdia/mdhd,/moov/trak/mdia/hdlr,/moov/trak/mdia/minf,/moov/trak/mdia/minf/vmhd,/moov/trak/mdia/minf/dinf,/moov/trak/mdia/minf/dinf/dref,/moov/trak/mdia/minf/stbl,/moov/trak/mdia/minf/stbl/stsd,/moov/trak/mdia/minf/stbl/stts,/moov/trak/mdia/minf/stbl/stsc,/moov/trak/mdia/minf/stbl/stsz,/moov/trak/mdia/minf/stbl/stco,/moov/trak/mdia/minf/stbl/stss,/moov/trak/mdia/minf/smhd,/moov/trak/mdia/minf/stbl/esds,/moov/trak/mdia/minf/stbl/ctts,/moov/udta,/free,/moov/trak/edts,/moov/trak/mdia/minf/stbl/sgpd,/moov/trak/mdia/minf/stbl/sbgp,/smss,/stsd,/stts,/stss,/ctts,/stsc,/stsz,/cprt,/,IsMal
0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1
1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
6,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
7,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0


# Atom names

In [ ]:
def get_atom_names(boxes):
  atoms = []
  for box in boxes:
    if hasattr(box, 'type'):
      atoms.append(box.type.decode())
    if hasattr(box, 'children') and box.children:
      atoms += get_atom_names(box.children)

  return atoms

def update_atom_names(atom_names, boxes):
  atoms = atom_names
  for box in boxes:
    if hasattr(box, 'type'):
      if box.type.decode() not in atoms:
        atoms.append(box.type.decode())
    if hasattr(box, 'children') and box.children:
      res = get_atom_names(box.children)
      for a in res:
        if a not in atoms:
          atoms.append(a)
  return atoms

In [ ]:
def atom_names_FE(file_name, IsMal, all_features, representation='count'):
  with open(file_name, 'rb') as f:
    boxes = parse_boxes(f)
    features = get_atom_names(boxes)  #Extract feature from file
  if representation == 'count':
    return [features.count(feature) for feature in all_features] + [IsMal]
  if representation == 'bin':
    return [int(feature in features) for feature in all_features] + [IsMal]
  return None

In [ ]:
## Create features set for dataset
atom_names = []

# Directory containing MP4 files
ok_directory = '/content/drive/MyDrive/nt230_collection/ok'
mal_directory = '/content/drive/MyDrive/nt230_collection/mal'

for filename in os.listdir(ok_directory):
    if filename.endswith('.mp4'):
        file_path = os.path.join(ok_directory, filename)
        # Updating the atom name feature set
        with open(file_path, 'rb') as f:
            boxes = parse_boxes(f)
            atom_names = update_atom_names(atom_names,boxes)
        print(f"Update atom names features set by {file_path}")
for filename in os.listdir(mal_directory):
    if filename.endswith('.mp4'):
        file_path = os.path.join(mal_directory, filename)
        with open(file_path, 'rb') as f:
            boxes = parse_boxes(f)
            atom_names = update_atom_names(atom_names,boxes)
        print(f"Update atom names features set by {file_path}")

# Create dataset from features set
dataset_an = pd.DataFrame(columns=atom_names + ['IsMal'])   # IsMal is label 1 is malware, 0 is not malware

# Update dataset
for filename in os.listdir(mal_directory):
    if filename.endswith('.mp4'):
        file_path = os.path.join(mal_directory, filename)
        dataset_an.loc[len(dataset_an)] = atom_names_FE(file_path, 1, atom_names)
for filename in os.listdir(ok_directory):
    if filename.endswith('.mp4'):
        file_path = os.path.join(ok_directory, filename)
        dataset_an.loc[len(dataset_an)] = atom_names_FE(file_path, 0, atom_names)

# Create dataset from features set
dataset_an_bin = pd.DataFrame(columns=atom_names + ['IsMal'])   # IsMal is label 1 is malware, 0 is not malware

# Update dataset
for filename in os.listdir(mal_directory):
    if filename.endswith('.mp4'):
        file_path = os.path.join(mal_directory, filename)
        dataset_an_bin.loc[len(dataset_an_bin)] = atom_names_FE(file_path, 1, atom_names, representation='bin')
for filename in os.listdir(ok_directory):
    if filename.endswith('.mp4'):
        file_path = os.path.join(ok_directory, filename)
        dataset_an_bin.loc[len(dataset_an_bin)] = atom_names_FE(file_path, 0, atom_names, representation='bin')

Update atom names features set by /content/drive/MyDrive/nt230_collection/ok/ok.mp4
Update atom names features set by /content/drive/MyDrive/nt230_collection/ok/ok1.mp4
Update atom names features set by /content/drive/MyDrive/nt230_collection/ok/ok2.mp4
Update atom names features set by /content/drive/MyDrive/nt230_collection/ok/ok3.mp4
Update atom names features set by /content/drive/MyDrive/nt230_collection/mal/mal.mp4
Update atom names features set by /content/drive/MyDrive/nt230_collection/mal/cve-2015-1538-1.mp4
Update atom names features set by /content/drive/MyDrive/nt230_collection/mal/hevc-crash-poc.mp4
Update atom names features set by /content/drive/MyDrive/nt230_collection/mal/videopoc.mp4


In [ ]:
dataset_an.to_csv('dataset_4.csv', index=False)
dataset_an_bin.to_csv('dataset_5.csv', index=False)

In [ ]:
print(dataset_an.shape)
dataset_an

(8, 33)


,ftyp,uuid,mdat,moov,mvhd,trak,tkhd,mdia,mdhd,hdlr,minf,vmhd,dinf,dref,stbl,stsd,stts,stsc,stsz,stco,stss,smhd,esds,ctts,udta,free,edts,sgpd,sbgp,smss,cprt,,IsMal
0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,0,0,0,0,0,1,1,1,1
1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,1,1,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1,1,1,1,1,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,1,1,1,1,1,0,0,0,0,0,0,0,0
5,1,0,1,1,1,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,1,1,1,0,1,1,2,1,1,0,0,0,0
6,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0
7,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0


In [ ]:
dataset_an_bin

,ftyp,uuid,mdat,moov,mvhd,trak,tkhd,mdia,mdhd,hdlr,minf,vmhd,dinf,dref,stbl,stsd,stts,stsc,stsz,stco,stss,smhd,esds,ctts,udta,free,edts,sgpd,sbgp,smss,cprt,,IsMal
0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,0,0,0,0,0,1,1,1,1
1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,1,1,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
5,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,0,0,0
6,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0
7,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0
